In [ ]:
import os
import pickle
import shutil
import sys
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from topostats.io import LoadScans

today = datetime.today().strftime("D-%Y-%m-%d-T-%H-%M")
print(today)

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/beaks")
assert base_dir.exists()
beak_topo_data_dir = base_dir / "output-beaks-topostats-unet-good"
assert beak_topo_data_dir.exists()

hummingbird_dir = beak_topo_data_dir / "hummingbird/processed/"
assert hummingbird_dir.exists()
magpie_dir = beak_topo_data_dir / "magpie/processed/"
assert magpie_dir.exists()

# grab files in both directories ending in .topostats
hummingbird_files = hummingbird_dir.glob("*.topostats")
magpie_files = magpie_dir.glob("*.topostats")
# merge lists
all_files = list(hummingbird_files) + list(magpie_files)
print(f"Found {len(all_files)} files")

In [ ]:
file = all_files[0]

# load the file
loadscans = LoadScans([file], channel="dummy")
loadscans.get_data()
img_dict = loadscans.img_dict
data = img_dict[file.stem]

print(data.keys())

image = data["image"]
ordered_traces = data["ordered_traces"]
plt.imshow(image)

plt.show()